#Machine Learning
###Exemplo de modelo de regressão p/ previsão de preços de venda de carros usados
* Este tutorial foi inspirado no treinamento de Cientista de Dados do [MS Learn](https://docs.microsoft.com/en-us/learn/). Algumas partes foram traduzidas para facilitar o entendimento, porém sugiro fortemente que todos realizem o treinamento oficial para um aprendizado mais aprofundado das técnicas que serão demonstradas aqui.

Fique a vontade para utilizar e adaptar conforme sua necessidade (ou divertimento 😎).

### Neste tutorial faremos uma [regressão linear simples](https://pt.wikipedia.org/wiki/Regress%C3%A3o_linear_simples) em que o objetivo será prever o PREÇO (y) de venda de um determinado modelo de veículo se baseando em algumas variáveis de entrada (X) 

A base de dados utilizada pode ser obtida no [link](https://github.com/lfbraz/machine-learning-tutorial/blob/master/datasets/dataset-carros-usados.csv) e foi baseada na versão [original](https://databricksdemostore.blob.core.windows.net/data/02.02/UsedCars.csv) disponibilizada pela Databricks, em que foi adaptada e traduzida para Português-Brasil.

##Importar base de dados
Utilizarei o [Azure Databricks](https://azure.microsoft.com/pt-br/services/databricks/) para treino do modelo, porém este tutorial pode ser utilizado com qualquer plataforma (sendo necessário apenas que o método de importação seja adaptado).

Realizamos o download do dataset utilizando a biblioteca [`requests`](https://pypi.org/project/requests/).

In [4]:
import requests

filename = "dataset-carros-usados.csv"
url = "https://raw.githubusercontent.com/lfbraz/machine-learning-tutorial/master/datasets/{}".format(filename)
output_local_file_path = "/tmp/{}".format(filename)
output_dbfs_file_path = "/data/{}".format(filename)

print('Fazendo o download de: {} para o diretório {}'.format(url, output_local_file_path))

# Baixar e persistir o arquivo
file = requests.get(url)
open(output_local_file_path, 'wb').write(file.content)

# Copiar para a estrutura de arquivos do Databricks
dbutils.fs.cp("file:{}".format(output_local_file_path), "dbfs:{}".format(output_dbfs_file_path) )

print('Arquivo copiado de: {} para o diretório dbfs {}'.format(output_local_file_path, output_dbfs_file_path))

Fazendo o download de: https://raw.githubusercontent.com/lfbraz/machine-learning-tutorial/master/datasets/dataset-carros-usados.csv para o diretório /tmp/dataset-carros-usados.csv
Arquivo copiado de: /tmp/dataset-carros-usados.csv para o diretório dbfs /data/dataset-carros-usados.csv

##Ler o arquivo .csv em um Spark DataFrame
Por questões de performance, vamos ler o arquivo .csv utilizando um [DataFrame do Spark](https://spark.apache.org/docs/latest/sql-programming-guide.html).

Utilizaremos as segmentos opções: <br/>
<br/>
* format: CSV
* inferSchema: true / Permite que os tipos de dados sejam automaticamente inferidos
* header: true / Primeira linha será entendida como o cabeçalho
* sep: ";" / Será utilizado como delimitador de colunas o carácter ";"
* load: path / O caminho do arquivo que será carregado

In [6]:
file_type = "csv"
infer_schema = "true"
first_row_is_header = "true"
delimiter = ";"

carros_usados = spark.read.format(file_type) \
                     .option("inferSchema", infer_schema) \
                     .option("header", first_row_is_header) \
                     .option("sep", delimiter) \
                     .load(output_dbfs_file_path)

## Visualização dos Dados

A tabela de carros usados possui os seguintes campos:<br/>
<br/>
* **PRECO**: Preço de venda do veículo (variável target / previsão)
* **IDADE_ANOS**: Número de anos desde a data de fabricação do veículo
* **KM**: Número de kilometros rodados pelo veículo
* **TIPO_COMBUSTIVEL**: Tipo de combustível utilizado
* **[HP](https://en.wikipedia.org/wiki/Horsepower)**: Medida de potência do veículo
* **COR_METALICA**: Indica se o veículo possui cor metálica (1 para sim e 0 para não). Pode indicar maior valorização
* **AUTOMATICO**: Indica se o veículo é automático (1 para sim e 0 para não)
* **[CC](https://pt.wikipedia.org/wiki/Cilindrada)**: Cilindradas do veículo
* **QTD_PORTAS**: Número de portas do veículo
* **PESO_KG**: Peso em quilograma do veículo

Com o comando `display(carros_usados)` podemos analisar o DataFrame

In [8]:
carros_usados.show(n=5)

+-------+----------+--------+----------------+---+------------+----------+----+----------+-------+
 PRECO|IDADE_ANOS| KM|TIPO_COMBUSTIVEL| HP|COR_METALICA|AUTOMATICO| CC|QTD_PORTAS|PESO_KG|
+-------+----------+--------+----------------+---+------------+----------+----+----------+-------+
 7450.0| 65.0| 82000.0| GASOLINA| 86| 1| 0|1300| 3| 1015|
 7250.0| 74.0|130025.0| GASOLINA|110| 1| 0|1600| 3| 1050|
 8950.0| 80.0| 64000.0| GASOLINA|110| 0| 0|1600| 3| 1055|
11450.0| 54.0| 62987.0| GASOLINA|110| 0| 0|1600| 5| 1080|
 6950.0| 80.0| 62581.0| GASOLINA|110| 0| 0|1600| 5| 1075|
+-------+----------+--------+----------------+---+------------+----------+----+----------+-------+
only showing top 5 rows

Com o [Azure Databricks](https://azure.microsoft.com/pt-br/services/databricks/) assim que executamos o Display do DataFrame podemos na própria célula indicar os tipos de gráficos que queremos visualizar (clicando no icone gráfico abaixo do DataFrame e em `PlotOptions`).

Abaixo uma visualização por Tipo de Combustível e Preço (utilizando 30 `bins` [número de colunas] )

In [10]:
display(carros_usados)

PRECO,IDADE_ANOS,KM,TIPO_COMBUSTIVEL,HP,COR_METALICA,AUTOMATICO,CC,QTD_PORTAS,PESO_KG
7450.0,65.0,82000.0,GASOLINA,86,1,0,1300,3,1015
7250.0,74.0,130025.0,GASOLINA,110,1,0,1600,3,1050
8950.0,80.0,64000.0,GASOLINA,110,0,0,1600,3,1055
11450.0,54.0,62987.0,GASOLINA,110,0,0,1600,5,1080
null,42.0,38932.0,GASOLINA,110,1,0,1600,3,1040
6950.0,80.0,62581.0,GASOLINA,110,0,0,1600,5,1075
8250.0,70.0,59017.0,GASOLINA,107,1,1,1600,3,1080
12950.0,44.0,41499.0,GAS_NATURAL_COMPRIMIDO,110,1,0,1600,5,1103
9950.0,65.0,65513.0,GASOLINA,110,1,1,1600,4,1070
7900.0,75.0,125400.0,GASOLINA,110,0,0,1600,3,1050


Também podemos utilizar outras bibliotecas para análise gráfica como o `matplotlib` ou `seaborn`. Neste caso pode-se converter o DataFrame Spark para um DataFrame Pandas (com isso a performance pode ser degradada).

Abaixo um [histograma](https://pt.wikipedia.org/wiki/Histograma) gerado a partir do preço de venda do veículo utilizando o matplotlib a partir de um [DataFrame do Pandas](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html).

In [12]:
import matplotlib.pyplot as plt
%matplotlib inline 

display(carros_usados.toPandas().plot(kind='hist', y='PRECO'))

# Criando uma regressão linear simples

Podemos iniciar investigando se a idade do veículo pode influenciar seu preço de venda.

In [15]:
carros_usados_pd = carros_usados.toPandas()

fig, ax = plt.subplots()

# Populate the figure
plt.scatter(carros_usados_pd['IDADE_ANOS'], carros_usados_pd['PRECO'])

# Set various labels
plt.title('Preço dos carros usados como uma função da idade')
plt.ylabel('Preço [R$]')
plt.xlabel('Idade [Meses]')

# Extras?
plt.grid() # Turn plot-grid on

# Show figure
display(fig)

Repare que quanto maior a idade do veículo menor o seu preço de venda (make sense??).

Geralmente quando falamos em Machine Learning (aprendizado de máquina) estamos pensando em uma tarefa específica. No nosso caso, a tarefa básica 
é a de **prever o preço de venda de um veículo** utilizando para isso dados históricos de vendas de outros veículos.

Regressão Linear é uma das primeiras técnicas estatísticas aprendidas para previsão de dados que se comportam de forma linear 🙄. <br/><br/>
E o que isso significa ? Basicamente estamos falando que quando uma variável cresce (ou diminui) outra variável também têm o mesmo comportamento, como por exemplo, o **PREÇO** do veículo com a sua **IDADE** em meses (que acabamos de analisar de forma gráfica).

Este comportamento, para os mais entendidos, é feito através da famosa equação \\(y = ax + b\\).

No exemplo dado estamos falando apenas na relação de duas variáveis (PREÇO e IDADE), porém a ideia é utilizar diferentes variáveis no mesmo modelo linear.

Neste tipo de situação (que é a mais comum) a visualização é mais complexa pois a previsão não se dará por uma única reta (mas sim por hiperplanos 😳). Quem quiser entender mais sobre isso, recomendo o vídeo [Regressão Linear Múltipla](https://drive.google.com/file/d/1MKIO-oe8mtz92rlZp3eZJIYVmQSpW0Pi/view) de uma aula dada no IME/USP sobre o assunto.

## Tratamento dos dados
Antes de aplicarmos um modelo de Regressão Linear precisamos primeiramente tratar os dados que serão ENTRADA do modelo, isto é, precisaremos limpar, padronizar e enriquecer os dados que serão utilizados para treinamento do modelo.

In [18]:
carros_usados.show(n=5)

+-------+----------+--------+----------------+---+------------+----------+----+----------+-------+
 PRECO|IDADE_ANOS| KM|TIPO_COMBUSTIVEL| HP|COR_METALICA|AUTOMATICO| CC|QTD_PORTAS|PESO_KG|
+-------+----------+--------+----------------+---+------------+----------+----+----------+-------+
 7450.0| 65.0| 82000.0| GASOLINA| 86| 1| 0|1300| 3| 1015|
 7250.0| 74.0|130025.0| GASOLINA|110| 1| 0|1600| 3| 1050|
 8950.0| 80.0| 64000.0| GASOLINA|110| 0| 0|1600| 3| 1055|
11450.0| 54.0| 62987.0| GASOLINA|110| 0| 0|1600| 5| 1080|
 6950.0| 80.0| 62581.0| GASOLINA|110| 0| 0|1600| 5| 1075|
+-------+----------+--------+----------------+---+------------+----------+----+----------+-------+
only showing top 5 rows

O primeiro ponto importante é analisarmos se existem valores faltantes para cada uma das colunas que serão utilizadas no modelo. 

Repare que existem valores `null` (faltantes) no conjunto de dados analisado. Podemos checar estes valores através do comando abaixo:

In [20]:
from pyspark.sql.functions import isnan, when, col

for c in carros_usados.columns:
  carros_usados.where(col(c).isNull()).show()

+-----+----------+---+----------------+---+------------+----------+---+----------+-------+
PRECO|IDADE_ANOS| KM|TIPO_COMBUSTIVEL| HP|COR_METALICA|AUTOMATICO| CC|QTD_PORTAS|PESO_KG|
+-----+----------+---+----------------+---+------------+----------+---+----------+-------+
+-----+----------+---+----------------+---+------------+----------+---+----------+-------+

+-----+----------+---+----------------+---+------------+----------+---+----------+-------+
PRECO|IDADE_ANOS| KM|TIPO_COMBUSTIVEL| HP|COR_METALICA|AUTOMATICO| CC|QTD_PORTAS|PESO_KG|
+-----+----------+---+----------------+---+------------+----------+---+----------+-------+
+-----+----------+---+----------------+---+------------+----------+---+----------+-------+

+-----+----------+---+----------------+---+------------+----------+---+----------+-------+
PRECO|IDADE_ANOS| KM|TIPO_COMBUSTIVEL| HP|COR_METALICA|AUTOMATICO| CC|QTD_PORTAS|PESO_KG|
+-----+----------+---+----------------+---+------------+----------+---+----------+-------+
+-----+----------+---+----------------+---+------------+----------+---+----------+-------+

+-----+----------+---+----------------+---+------------+----------+---+----------+-------+
PRECO|IDADE_ANOS| KM|TIPO_COMBUSTIVEL| HP|COR_METALICA|AUTOMATICO| CC|QTD_PORTAS|PESO_KG|
+-----+----------+---+----------------+---+------------+----------+---+----------+-------+
+-----+----------+---+----------------+---+------------+----------+---+----------+-------+

+-----+----------+---+----------------+---+------------+----------+---+----------+-------+
PRECO|IDADE_ANOS| KM|TIPO_COMBUSTIVEL| HP|COR_METALICA|AUTOMATICO| CC|QTD_PORTAS|PESO_KG|
+-----+----------+---+----------------+---+------------+----------+---+----------+-------+
+-----+----------+---+----------------+---+------------+----------+---+----------+-------+

+-----+----------+---+----------------+---+------------+----------+---+----------+-------+
PRECO|IDADE_ANOS| KM|TIPO_COMBUSTIVEL| HP|COR_METALICA|AUTOMATICO| CC|QTD_PORTAS|PESO_KG|
+-----+----------+---+----------------+---+------------+----------+---+----------+-------+
+-----+----------+---+----------------+---+------------+----------+---+----------+-------+

+-----+----------+---+----------------+---+------------+----------+---+----------+-------+
PRECO|IDADE_ANOS| KM|TIPO_COMBUSTIVEL| HP|COR_METALICA|AUTOMATICO| CC|QTD_PORTAS|PESO_KG|
+-----+----------+---+----------------+---+------------+----------+---+----------+-------+
+-----+----------+---+----------------+---+------------+----------+---+----------+-------+

+-----+----------+---+----------------+---+------------+----------+---+----------+-------+
PRECO|IDADE_ANOS| KM|TIPO_COMBUSTIVEL| HP|COR_METALICA|AUTOMATICO| CC|QTD_PORTAS|PESO_KG|
+-----+----------+---+----------------+---+------------+----------+---+----------+-------+
+-----+----------+---+----------------+---+------------+----------+---+----------+-------+

+-----+----------+---+----------------+---+------------+----------+---+----------+-------+
PRECO|IDADE_ANOS| KM|TIPO_COMBUSTIVEL| HP|COR_METALICA|AUTOMATICO| CC|QTD_PORTAS|PESO_KG|
+-----+----------+---+----------------+---+------------+----------+---+----------+-------+
+-----+----------+---+----------------+---+------------+----------+---+----------+-------+

+-----+----------+---+----------------+---+------------+----------+---+----------+-------+
PRECO|IDADE_ANOS| KM|TIPO_COMBUSTIVEL| HP|COR_METALICA|AUTOMATICO| CC|QTD_PORTAS|PESO_KG|
+-----+----------+---+----------------+---+------------+----------+---+----------+-------+
+-----+----------+---+----------------+---+------------+----------+---+----------+-------+

Existem diversas técnicas de tratamento de valores faltantes (substituição pela média, moda, etc). Para simplificarmos, o processo vamos simplesmente remover toda a linha em que seja encontrado algum valor faltante.

In [22]:
carros_usados = carros_usados.na.drop()

Outro ponto importante é analisarmos o *TIPO* das variáveis de entrada. Repare que a variável **TIPO_COMBUSTIVEL** é uma variável categórica em formato TEXTO e quando falamos em modelos de aprendizado de máquina precisamos que todas as variáveis de *INPUT* sejam de alguma forma retratadas de forma numérica. Desta forma, o modelo (que nada mais é do que uma expressão matemática, muitas vezes extremamente complexa) conseguirá utilizar os dados de forma apropriada.

Para tratamento da variável utilizaremos as técnicas `StringIndexer` e `OneHotEncoder` que permitem representar as varíaveis categóricas em um formato vetorial binário. 

Com a `StringIndexer` representaremos as varíaveis categóricas de forma numérica (ex: GASOLINA=0, DIESEL=1, etc).

In [25]:
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler

indexer = StringIndexer(inputCol='TIPO_COMBUSTIVEL', outputCol='TIPO_COMBUSTIVEL_index')
indexed = indexer.fit(carros_usados).transform(carros_usados)
indexed.show()

+-------+----------+--------+--------------------+---+------------+----------+----+----------+-------+----------------------+
 PRECO|IDADE_ANOS| KM| TIPO_COMBUSTIVEL| HP|COR_METALICA|AUTOMATICO| CC|QTD_PORTAS|PESO_KG|TIPO_COMBUSTIVEL_index|
+-------+----------+--------+--------------------+---+------------+----------+----+----------+-------+----------------------+
 7450.0| 65.0| 82000.0| GASOLINA| 86| 1| 0|1300| 3| 1015| 0.0|
 7250.0| 74.0|130025.0| GASOLINA|110| 1| 0|1600| 3| 1050| 0.0|
 8950.0| 80.0| 64000.0| GASOLINA|110| 0| 0|1600| 3| 1055| 0.0|
11450.0| 54.0| 62987.0| GASOLINA|110| 0| 0|1600| 5| 1080| 0.0|
 6950.0| 80.0| 62581.0| GASOLINA|110| 0| 0|1600| 5| 1075| 0.0|
 8250.0| 70.0| 59017.0| GASOLINA|107| 1| 1|1600| 3| 1080| 0.0|
12950.0| 44.0| 41499.0|GAS_NATURAL_COMPR...|110| 1| 0|1600| 5| 1103| 2.0|
 9950.0| 65.0| 65513.0| GASOLINA|110| 1| 1|1600| 4| 1070| 0.0|
 7900.0| 75.0|125400.0| GASOLINA|110| 0| 0|1600| 3| 1050| 0.0|
 6495.0| 74.0| 96302.0| GASOLINA| 86| 1| 0|1300| 3| 1015| 0.0|
 9500.0| 66.0| 74963.0| GASOLINA|107| 0| 1|1600| 3| 1085| 0.0|
 6490.0| 80.0|100123.0| GASOLINA|110| 1| 0|1600| 3| 1050| 0.0|
 7250.0| 80.0|110887.0| GASOLINA|110| 1| 0|1600| 3| 1055| 0.0|
 9450.0| 80.0| 66843.0| GASOLINA|110| 1| 0|1600| 5| 1075| 0.0|
16500.0| 27.0| 37177.0| GASOLINA|110| 0| 0|1600| 5| 1130| 0.0|
 9250.0| 63.0|102807.0| GASOLINA| 86| 0| 0|1300| 5| 1055| 0.0|
 6900.0| 69.0|146304.0| DIESEL| 72| 1| 0|2000| 3| 1115| 1.0|
11950.0| 56.0| 33998.0| GASOLINA|110| 0| 0|1600| 5| 1080| 0.0|
 9950.0| 61.0| 97425.0| GASOLINA|110| 1| 0|1600| 5| 1075| 0.0|
 8450.0| 69.0| 74573.0| GASOLINA|110| 1| 0|1600| 5| 1085| 0.0|
+-------+----------+--------+--------------------+---+------------+----------+----+----------+-------+----------------------+
only showing top 20 rows

Repare que foi adicionada uma nova coluna chamada **TIPO_COMBUSTIVEL_index** com a representação numérica mencionada.

Também vamos utilizar a técnica de `OneHotEncoder` para transformação vetorial binária da variável que foi indexada (uma lista mais completa das técnicas pode ser encontrada no [link](https://spark.apache.org/docs/latest/ml-features.html) ). Para otimização deste processo ainda não vamos executar as transformações, vamos criar `Stages` (estágios de processamento das transformações) que serão posteriormente colocadas em um `Pipeline`).

In [27]:
encoder = OneHotEncoderEstimator(inputCols=["TIPO_COMBUSTIVEL_index"],
                                 outputCols=["TIPO_COMBUSTIVEL_vetor"])

stages = [indexer, encoder]

A variável `stages` contém os estágios de `StringIndexer` e `OneHotEncoder`, além deles utilizaremos também a técnica de `VectorAssembler` para consolidar todas as variáveis (features) que serão utilizadas para treinamento do modelo e adicionaremos mais este estágio na variável `stages`.

In [29]:
colunas_treino = ['IDADE_ANOS', 'KM', 'HP', 'COR_METALICA', 'AUTOMATICO', 'CC', 'QTD_PORTAS', 'PESO_KG', 'TIPO_COMBUSTIVEL_vetor']

assembler = VectorAssembler(inputCols=colunas_treino, outputCol="features")
stages += [assembler]

Com os estágios de tratamento já definidos podemos agora aplicar as transformações em um [`Pipeline`](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html?highlight=onehotencoder#pyspark.ml.Pipeline).

In [31]:
from pyspark.ml import Pipeline
  
partialPipeline = Pipeline().setStages(stages)
pipelineModel = partialPipeline.fit(carros_usados)
DF_preparado = pipelineModel.transform(carros_usados)

In [32]:
display(DF_preparado)

PRECO,IDADE_ANOS,KM,TIPO_COMBUSTIVEL,HP,COR_METALICA,AUTOMATICO,CC,QTD_PORTAS,PESO_KG,TIPO_COMBUSTIVEL_index,TIPO_COMBUSTIVEL_vetor,features
7450.0,65.0,82000.0,GASOLINA,86,1,0,1300,3,1015,0.0,"List(0, 3, List(0), List(1.0))","List(1, 11, List(), List(65.0, 82000.0, 86.0, 1.0, 0.0, 1300.0, 3.0, 1015.0, 1.0, 0.0, 0.0))"
7250.0,74.0,130025.0,GASOLINA,110,1,0,1600,3,1050,0.0,"List(0, 3, List(0), List(1.0))","List(1, 11, List(), List(74.0, 130025.0, 110.0, 1.0, 0.0, 1600.0, 3.0, 1050.0, 1.0, 0.0, 0.0))"
8950.0,80.0,64000.0,GASOLINA,110,0,0,1600,3,1055,0.0,"List(0, 3, List(0), List(1.0))","List(1, 11, List(), List(80.0, 64000.0, 110.0, 0.0, 0.0, 1600.0, 3.0, 1055.0, 1.0, 0.0, 0.0))"
11450.0,54.0,62987.0,GASOLINA,110,0,0,1600,5,1080,0.0,"List(0, 3, List(0), List(1.0))","List(1, 11, List(), List(54.0, 62987.0, 110.0, 0.0, 0.0, 1600.0, 5.0, 1080.0, 1.0, 0.0, 0.0))"
6950.0,80.0,62581.0,GASOLINA,110,0,0,1600,5,1075,0.0,"List(0, 3, List(0), List(1.0))","List(1, 11, List(), List(80.0, 62581.0, 110.0, 0.0, 0.0, 1600.0, 5.0, 1075.0, 1.0, 0.0, 0.0))"
8250.0,70.0,59017.0,GASOLINA,107,1,1,1600,3,1080,0.0,"List(0, 3, List(0), List(1.0))","List(1, 11, List(), List(70.0, 59017.0, 107.0, 1.0, 1.0, 1600.0, 3.0, 1080.0, 1.0, 0.0, 0.0))"
12950.0,44.0,41499.0,GAS_NATURAL_COMPRIMIDO,110,1,0,1600,5,1103,2.0,"List(0, 3, List(2), List(1.0))","List(1, 11, List(), List(44.0, 41499.0, 110.0, 1.0, 0.0, 1600.0, 5.0, 1103.0, 0.0, 0.0, 1.0))"
9950.0,65.0,65513.0,GASOLINA,110,1,1,1600,4,1070,0.0,"List(0, 3, List(0), List(1.0))","List(1, 11, List(), List(65.0, 65513.0, 110.0, 1.0, 1.0, 1600.0, 4.0, 1070.0, 1.0, 0.0, 0.0))"
7900.0,75.0,125400.0,GASOLINA,110,0,0,1600,3,1050,0.0,"List(0, 3, List(0), List(1.0))","List(1, 11, List(), List(75.0, 125400.0, 110.0, 0.0, 0.0, 1600.0, 3.0, 1050.0, 1.0, 0.0, 0.0))"
6495.0,74.0,96302.0,GASOLINA,86,1,0,1300,3,1015,0.0,"List(0, 3, List(0), List(1.0))","List(1, 11, List(), List(74.0, 96302.0, 86.0, 1.0, 0.0, 1300.0, 3.0, 1015.0, 1.0, 0.0, 0.0))"


Em conjunto com as outras variáveis do DataFrame temos agora a coluna `features` que contém um consolidado vetorial de todas as variáveis que devem ser utilizadas para treinamento do modelo (já com as transformações realizadas em um Pipeline).

Com os dados preparados devemos agora dividir o conjunto de dados entre dados de treino (utilizado para treinar o modelo) e testes (utilizado para avaliar a performance do modelo). Utilizaremos a proporção de 80% para treino e 20% para teste.

In [35]:
treino, teste = DF_preparado.randomSplit([0.8, 0.2])


Com isso, finalmente 😆 podemos então realizar o treinamento de um modelo de Regressão Linear Simples.

In [37]:
from pyspark.ml.regression import LinearRegression


model = LinearRegression(featuresCol = 'features', labelCol='PRECO').fit(treino)

Com o modelo treinado podemos analisar as métricas de Erro quadrático médio [RSME](https://en.wikipedia.org/wiki/Root-mean-square_deviation) e o [R²](https://en.wikipedia.org/wiki/Coefficient_of_determination) para verificarmos a qualidade do estimador utilizado.

In [39]:
relatorio_treino = model.summary

print("RMSE: %f" % relatorio_treino.rootMeanSquaredError)
print("r2: %f" % relatorio_treino.r2)

RMSE: 1300.330689
r2: 0.877501

Agora podemos também utilizar o modelo para fazer as predições na base de teste

## Predições na base TESTE

In [42]:
predicoes = model.transform(teste)

display(predicoes)

PRECO,IDADE_ANOS,KM,TIPO_COMBUSTIVEL,HP,COR_METALICA,AUTOMATICO,CC,QTD_PORTAS,PESO_KG,TIPO_COMBUSTIVEL_index,TIPO_COMBUSTIVEL_vetor,features,prediction
4350.0,44.0,158320.0,DIESEL,69,0,0,1800,5,1110,1.0,"List(0, 3, List(1), List(1.0))","List(1, 11, List(), List(44.0, 158320.0, 69.0, 0.0, 0.0, 1800.0, 5.0, 1110.0, 0.0, 1.0, 0.0))",10911.823491642692
5250.0,72.0,126478.0,GASOLINA,110,1,0,1600,4,1035,0.0,"List(0, 3, List(0), List(1.0))","List(1, 11, List(), List(72.0, 126478.0, 110.0, 1.0, 0.0, 1600.0, 4.0, 1035.0, 1.0, 0.0, 0.0))",7243.84460590187
5250.0,79.0,207114.0,GAS_NATURAL_COMPRIMIDO,110,1,0,1600,5,1114,2.0,"List(0, 3, List(2), List(1.0))","List(1, 11, List(), List(79.0, 207114.0, 110.0, 1.0, 0.0, 1600.0, 5.0, 1114.0, 0.0, 0.0, 1.0))",5356.62127571641
5600.0,75.0,146197.0,GASOLINA,86,0,0,1300,3,1015,0.0,"List(0, 3, List(0), List(1.0))","List(1, 11, List(), List(75.0, 146197.0, 86.0, 0.0, 0.0, 1300.0, 3.0, 1015.0, 1.0, 0.0, 0.0))",5897.717018001891
5750.0,76.0,183277.0,DIESEL,72,0,0,2000,3,1115,1.0,"List(0, 3, List(1), List(1.0))","List(1, 11, List(), List(76.0, 183277.0, 72.0, 0.0, 0.0, 2000.0, 3.0, 1115.0, 0.0, 1.0, 0.0))",5727.221229920016
5900.0,73.0,232940.0,DIESEL,72,0,0,2000,5,1172,1.0,"List(0, 3, List(1), List(1.0))","List(1, 11, List(), List(73.0, 232940.0, 72.0, 0.0, 0.0, 2000.0, 5.0, 1172.0, 0.0, 1.0, 0.0))",6367.804093363773
5950.0,72.0,105856.0,GASOLINA,86,0,0,1300,3,1015,0.0,"List(0, 3, List(0), List(1.0))","List(1, 11, List(), List(72.0, 105856.0, 86.0, 0.0, 0.0, 1300.0, 3.0, 1015.0, 1.0, 0.0, 0.0))",6906.7215576614835
5950.0,75.0,74567.0,GASOLINA,86,1,0,1300,3,1015,0.0,"List(0, 3, List(0), List(1.0))","List(1, 11, List(), List(75.0, 74567.0, 86.0, 1.0, 0.0, 1300.0, 3.0, 1015.0, 1.0, 0.0, 0.0))",7069.399955748995
6150.0,80.0,194765.0,DIESEL,72,0,0,2000,3,1120,1.0,"List(0, 3, List(1), List(1.0))","List(1, 11, List(), List(80.0, 194765.0, 72.0, 0.0, 0.0, 2000.0, 3.0, 1120.0, 0.0, 1.0, 0.0))",5129.7040218882885
6250.0,80.0,92830.0,GASOLINA,86,1,0,1300,3,1015,0.0,"List(0, 3, List(0), List(1.0))","List(1, 11, List(), List(80.0, 92830.0, 86.0, 1.0, 0.0, 1300.0, 3.0, 1015.0, 1.0, 0.0, 0.0))",6153.158490462232


## Podemos agora analisar os resultados do RMSE e R² das predições

In [44]:
test_result = model.evaluate(teste)
print("RMSE = %g" % test_result.rootMeanSquaredError)
print("r2 = %g" % test_result.r2)

RMSE = 1371.93
r2 = 0.821776

### Podemos comparar graficamente o "ERRO" das predições realizadas com relação ao valor real de PREÇO.

In [46]:
from sklearn import metrics

fig, ax = plt.subplots()

y_pred = predicoes.toPandas().prediction
y_test = predicoes.toPandas().PRECO

# Make a list of all the errors in the test-dataset:
errors = (y_pred - y_test)

### Populate the figure
# Plot the test-data:
plt.scatter(teste.toPandas().PRECO, errors, color='red', edgecolors='black')

# Set various labels
plt.ylabel('ERROS')
plt.xlabel('PREÇO')

plt.title('Erros em $ por Preço')

# Extras?
plt.grid() # Turn plot-grid on
plt.legend()

# Show figure
display(fig)

##Persistir o modelo

Para ser possível a reutilização do modelo treinado podemos persisti-lo para que possa ser carregado posteriormente (sem a necessidade de re-treinamento).

Na Azure podemos utilizar o [ADLS (Azure Data Lake Storage)](https://azure.microsoft.com/en-us/services/storage/data-lake-storage/) para persistir os modelos gerados em containers criados no Data Lake. Para isso, podemos "montar" o container utilizando uma chave de criptografia, fazendo com que o ADLS apareça como uma nova pasta no Databricks. No [link](https://github.com/lfbraz/azure-databricks/blob/master/notebooks/read-from-adls.ipynb) temos mais explicações de como realizar este processo. 

Neste tutorial vamos persistir o modelo para um diretório previamente montado. Caso você não esteja utilizando o Databricks, basta utilizar um diretório local (ou equivalente a plataforma que estiver utilizando).

In [49]:
NOME_MODELO = 'modelo_regressao_linear.model'
model.save('/mnt/models/{}'.format(NOME_MODELO))